# Autoencoder und Generative Models 

Diese Kapitel gibt eine kurze Einführung in aktuelle Forschungsaktivitäten im Bereich des Deep Learning. Wir stellen dabei die beiden Themen 

- Autoencoder 
- Generative Models am Beispiel der Variational Autoencoder 

vor. Zur Behandlung beider Themen empfiehlt sich die sogannnte Functional API (Model aus keras.models). In dieser API können Layer als Funktionen (daher der Name Functional API) verwendet werden, die als Eingabe und Ausgabe Tensoren besitzen. Die Functional API erlaubt es zudem Tensoren auch innerhalb des Netzwerkes auszuwerten.

### Functional API in Keras

Mit der Functional API von Keras können layer als Funktionen aufgefasst werden, die als Input Tensoren erhalten und als Output Tensoren  zurückgeben. Die Functional API findet sich unter keras.models als Model:

https://keras.io/guides/functional_api/

Wir schauen dieses Vorgehen am Beispiel eines neuronalen Netzes mit zwei Hidden-Layer an (zur Illustration bauen wir diese Netz nochmals als Sequential model auf).

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import keras

from keras.models import Sequential # Sequential model API
from keras.models import Model      # Functional model API

from keras.layers import Dense, Conv2D, Conv2DTranspose, Flatten, Reshape, Lambda

from keras import Input
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
# Neuronales Netz mit zwei Zwischenschichten
n  = 784
h1 = 100
h2 = 100
K = 10

# Sequential model API
model1 = Sequential()
model1.add(Dense(h1,activation='relu',input_shape=(784,)))
model1.add(Dense(h2,activation='relu'))
model1.add(Dense(K,activation='softmax'))
#model1.summary()

# Functional model API
In  = Input(shape=(784,))
H1  = Dense(h1,activation='relu')(In)
H2  = Dense(h2,activation='relu')(H1)
Out = Dense(K,activation='softmax')(H2)

model2 = Model(In,Out)
model2.summary()

Sind wir beispielsweise an der Datenrepräsentation $H_2$ nach dem Trainig interresiert, so können wir mir der Functional API ein Netz erstellen, welches die inputs "In" auf $H_2$ abbildet. Bezeichnen wir dieses Netz als coding, so ergibt sich:

In [ ]:
coding = Model(In,H2)
coding.summary()

Wir trainieren das model2 mit den MNIST Daten.

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(60000,28*28 )
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(10000,28*28 )

In [ ]:
model2.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',metrics=['acc'])
model2.fit(x_train,y_train,epochs=10,batch_size=100)

In [ ]:
# Auslesen von H2
coding.predict(x_test).shape

### Autoencoder

Sogenannte **Autoencoder** zählen zu den unsupervised Learning Algorithmen. Sie bestehen aus künstlichen neuronales Netzen, welche u.a. dazu verwendet werden, effiziente Codierungen zu lernen. Ein Autoencoders hat zum Ziel, eine komprimierte Repräsentation (sogenanntes Encoding) für einen Datensatz zu lernen und somit wesentliche Merkmale zu extrahieren (sogenannte Feature Extraction). 

Das übliche Vorgehen bei Autoencodern ist wie folgt

- Man bildet einen (oft hochdimensionalen) Input mittels einer oder mehrerer Zwischenschichten (diese können auch aus Convolutional Layern bestehen) auf einen niederdimensionalen Zwischenlayer ab (dieser enthält das oben angesprochene Encoding des Inputs). Diese Teil des neuronalen Netzes wird **Encoder** genannt.
- Ausgehend von diesem Encoding wird mittels einer oder mehrerer Zwischenschichten wieder auf den Output abgebildet. Dieser Teil des Netzes wird **Decoder** genannt. 

Beide Vorgänge sind in der folgenden Abbildung illustriert:

<img src="autoencoder.png" height="100" width="600"/>

Bestehen Encoder/Decoder aus vielen (Convolutional) Layern, so spricht man von **Deep Autoencoder**.
Die Cosfunction eines Autoencoders ist im einfachen Falle wie folgt konstruiert:
$$
J(\mathbf{W}) = \frac{1}{2m}\sum_{i=1}^m\left(\vec{x}^{(i)}-\vec{x}_{p}^{(i)}\right)^2
$$
Hierbei ist $\vec{x}^{(i)}$ der Input von Training Example $i$ (in der Illustration oben die das vektorisierte Bild der Zahl 4 links). $\vec{x}_{p}^{(i)}$ bezeichnet den Output des  Encoder/Decoder-Netzwerkes (in der Illustration oben die das vektorisierte Bild der Zahl 4 rechts). Dieser Output hängt von der Gesamtheit aller Gewichte $\mathbf{W}$ ab.

#### Erstellung eines einfachen Autoencoders

In [ ]:
# Encoder 
In  = Input(shape=(784,))
z  = Dense(2)(In)

encoder = Model(In,z)

# Decoder 
dec_In = Input(K.int_shape(z)[1:])
Out = Dense(784)(dec_In)

decoder = Model(dec_In,Out)

z_decoded = decoder(z)

model = Model(In,z_decoded)
#model.summary()

encoder.summary()
decoder.summary()
#model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
#model.fit(x_train,x_train,epochs=10,batch_size=50)

In [ ]:
In  = Input(shape=(784,))
z   = Dense(100)(In)
Out = Dense(784)(z) 


model = Model(In,Out)
#model.summary()
encoder = Model(In,z)
#encoder.summary()
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.fit(x_train,x_train,epochs=10,batch_size=50)

### Visualisierung Latent Space

In [ ]:
plt.scatter(encoder.predict(x_test)[:,2],encoder.predict(x_test)[:,80],c=y_test,cmap='jet')
plt.colorbar()

In [ ]:
encoder.predict(x_test)

In [ ]:
L = 2
plt.imshow(model.predict(x_test)[L,:].reshape(28,28),cmap='gray')
plt.show()
plt.imshow(x_test[L,:].reshape(28,28),cmap='gray')

### Reconstruction

In [ ]:
L = 1
plt.subplot(1,2,1)
plt.imshow(model.predict(x_test)[L,:].reshape(28,28),cmap='gray')
plt.subplot(1,2,2)
plt.imshow(x_train[L,:].reshape(28,28),cmap='gray')

### Erstellung eines Deep Autoencoders

Obig vorgestellter einfacher Ansatz eines Autoencoders führt im Falle einer linearen Aktivierung in der Zwischenschicht zu einer Principal Component Analysis. Wir stellen fest: Hat das Encoding nur zwei Dimensionen, so ist die Rekonstruktion sehr schlecht (d.h. $\vec{x}^{(i)}_p$ weicht stark von $\vec{x}^{(i)}$ ab). Es würden hier relativ viele Dimensionen für das Encoding benötigt, um den Rekonstruktion wieder korrekt zu erstellen. Im folgenden wollen wir den Encoder/Decoder daher verbessern, indem wir ein tiefe neuronale Netze mit Convolutional Layern verwenden.

In [3]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

In [ ]:
img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2  


# ENCODER-
input_img = Input(shape=img_shape)

x = Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)

shape_before_flattening = K.int_shape(x)
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
z = Dense(2)(x)

encoder = Model(input_img,z)

# DECODER
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(np.prod(shape_before_flattening[1:]),activation='relu')(decoder_input)
x = Reshape(shape_before_flattening[1:])(x)
x = Conv2DTranspose(32, 3,padding='same', activation='relu', strides=(2, 2))(x)
out = Conv2D(1, 3,padding='same', activation='sigmoid')(x)

decoder = Model(decoder_input, out)
z_decoded = decoder(z)

dae = Model(input_img, z_decoded)
dae.summary()
decoder.summary()
# Either train ..
#dae.compile(optimizer='rmsprop', loss='mse')
#dae.fit(x_train,x_train,epochs=10,batch_size=50)
#dae.save_weights('dae_weights_2.h5')

#... oder load weight
dae.load_weights('dae_weights.h5')

In [ ]:
x_test.shape

Der (in diesem Falle zweidimensionale Vektor-) Raum, welcher die Encodings der Inputs enthält, wird auch als
**Latent Space** bezeichnet. Wir visualisieren diesen, indem wir den Encoder-Teil der Netzwerkarchitektur
auf die Test-Daten anwenden. Wir erhalten folgenden Latent Space:

In [ ]:
latent_space = encoder.predict(x_test)
plt.scatter(latent_space[:,0],latent_space[:,1],c=y_test,cmap='jet')
plt.colorbar()

In [ ]:
x_train[L,:].shape

In [ ]:
L = 1
plt.subplot(1,2,1)
plt.imshow(dae.predict(x_train[L,:].reshape(1,28,28,1)).reshape(28,28),cmap='gray')
plt.subplot(1,2,2)
plt.imshow(x_train[L,:].reshape(28,28),cmap='gray')

Obwohl der Latent space nun (auch im Vergleich zum obigen Beispiel eines einfaches Autoencoders) sehr niederdimesional ist und eine sehr gute Rekonstruction erlaubt, hat er dennoch auch Nachteile: der Latent Space
ist unzusammendhängend und strukturlos. Diesen Umstand versucht ein sogenannter Variational Autoencoder zu berichtigen.

### Variational Autoencoder

Die **Variational Autoencoder** zählen zur Klasse der sogenannten **Generative Models**: Modelle, sie erlauben, ausgehend von einem zusammenhängen und strukturierten Latent Space entweder neue Bilder zu erzeugen bzw. bestehende Bilder gezielt zu verändern. 

<img src="variational_autoencoder.png" height="100" width="600"/>

Folgende Schritte werden bei einem VAE durchgeführt

(1) Der Input wird codiert in einen Mittelwert $\mu$ und Standartabweichung $\sigma$

$$
\vec{\mu}, \vec{\sigma} = \mathrm{encoder}(\mathrm{input\_img})
$$

(2) Von der Gaußverteilung $g(\mu,\sigma)$ wird dann zufällig Vektor $\vec{z}$ des Latent Spaces ausgewählt:

$$
\vec{z} = \vec{\mu} + \vec{\sigma} \cdot \vec{\epsilon}
$$

(3) Dieser Vektor $\vec{z}$ wird dann zurücktransformiert

$$
\mathrm{reconstructed\_img} = \mathrm{decoder}(\vec{z})
$$

(4) Wie wüblich wird ein Model erstellt

$$
\mathrm{model} = \mathrm{Model}(\mathrm{input\_img}, \mathrm{reconstructed\_img})
$$

(5) Dieses Model wird mit dann mit folgender Costfunction (bestehend aus reconstruction loss and regularization loss) trainiert

$$
J(\mathbf{W}) = 
\frac{1}{2m}\sum_{i=1}^m\left(\vec{x}^{(i)}-\vec{x}_{p}^{(i)}\right)^2
+\sum_{i=1}^{\mathrm{latent\_dim}}\mu_i^2+\sigma_i^2-\log(\sigma_i^2)-1 
$$

Diese Costfunction besteht aus zwei Teilen:
- Der erste Anteil versucht (wie schon bei den Autoencodern) den Input möglichst gut auf den Output zu matchen
- Der zweite Anteil versucht, dass jedes Trainingsbespiel nicht von **einem** Punkt des Latent Spaces entsteht, sondern von einer Gaußverteilung mit $\vec{\mu}=0$ und $\vec{\sigma}=1$. Dieser Anteil versucht eine möglichst hohe Struktur in den Latent Space zu bringen.


$$
J(\mathbf{W}) = 
\frac{1}{2m}\sum_{i=1}^m\left(\vec{x}^{(i)}-\vec{x}_{p}^{(i)}\right)^2
+\sum_{i=1}^{\mathrm{latent\_dim}}\mu_i^2+\sigma_i^2-\log(\sigma_i^2)-1 
$$


In [4]:
img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2  

#########
# ENCODER
#########
input_img = Input(shape=img_shape)

x = Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
shape_before_flattening = K.int_shape(x)
x = Flatten()(x)
x = Dense(32, activation='relu')(x)

z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])


#########
# DECODER
#########
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(np.prod(shape_before_flattening[1:]),activation='relu')(decoder_input)
x = Reshape(shape_before_flattening[1:])(x)
x = Conv2DTranspose(32, 3,padding='same', activation='relu', strides=(2, 2))(x)
x = Conv2D(1, 3,padding='same', activation='sigmoid')(x)

decoder = Model(decoder_input, x)
z_decoded = decoder(z)

class CustomVariationalLayer(keras.layers.Layer):

    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
            1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

y = CustomVariationalLayer()([input_img, z_decoded])

vae = Model(input_img, y)
# Either train ...
#vae.compile(optimizer='rmsprop', loss=None)
#vae.fit(x=x_train, y=None,shuffle=True,epochs=10,batch_size=batch_size,validation_data=(x_test, None))
#vae.save_weights('va_weights.h5')

# ... oder load weights
vae.load_weights('vae_weights.h5')

W0709 12:55:37.855664 140338461648704 module_wrapper.py:139] From /opt/tljh/user/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0709 12:55:37.860863 140338461648704 module_wrapper.py:139] From /opt/tljh/user/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0709 12:55:37.867327 140338461648704 module_wrapper.py:139] From /opt/tljh/user/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0709 12:55:37.950902 140338461648704 module_wrapper.py:139] From /opt/tljh/user/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0709 12:55:38.035674 140338461648704 module_wrapper.py:1

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(decoder.predict(np.array([[-0.0,2.0]])).reshape(28,28),cmap='gray')

In [8]:
encoder = Model(input_img,z_mean)
x_test.shape

(10000, 28, 28, 1)

In [ ]:
mu_pred = encoder.predict(x_test[:,:].reshape(-1,28,28,1))

In [5]:
plt.scatter(mu_pred[:,0],mu_pred[:,1],c=y_test,cmap='jet')
plt.colorbar()

NameError: name 'mu_pred' is not defined

In [ ]:
keras.layers.Layer